In [9]:
import matplotlib.pyplot as plt
import s3fs
import pandas as pd
import numpy as np
from itertools import product
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split,train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score,mean_squared_error, mean_absolute_error, r2_score,confusion_matrix
from sklearn.preprocessing import MinMaxScaler
y_pred =pd.read_csv("imputedResultSample.csv")
y_test =pd.read_csv("originalTestSample.csv")
valid_pos = y_test.notna()

In [3]:
y_test

,Unnamed: 0.1,Unnamed: 0,0,1,2,3,4,5,6,7,8,9
0,0,11,4.3,137.0,102.0,38.4,1.0,20.0,19.0,19.0,77.0,154.0
1,1,12,3.2,141.0,103.0,38.2,1.1,31.0,10.0,22.0,156.0,146.0
2,2,13,3.4,145.0,103.0,39.0,1.0,30.0,15.0,18.0,99.0,159.0
3,3,55,4.0,138.0,103.0,37.8,0.8,28.0,11.0,13.0,108.0,172.0
4,4,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,50329,4.3,139.0,97.0,40.6,1.6,30.0,16.0,32.0,248.0,257.0
9996,9996,50361,3.8,140.0,109.0,34.2,0.9,22.0,13.0,15.0,143.0,174.0
9997,9997,50362,4.6,142.0,107.0,38.3,0.9,26.0,14.0,11.0,138.0,165.0
9998,9998,50363,4.8,139.0,104.0,35.4,0.8,28.0,12.0,7.0,125.0,104.0


In [2]:
y_pred

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10
0,0,11.000000,4.300000,137.000000,102.000000,38.400000,1.000000,20.000000,19.000000,20.567667,77.000000,154.000000
1,1,12.000000,3.200000,143.671295,103.000000,38.200000,1.093158,31.000000,13.218160,19.296064,121.255043,146.000000
2,2,13.000000,3.400000,145.000000,103.000000,39.000000,1.000000,30.000000,15.000000,18.000000,99.000000,159.000000
3,3,55.000000,4.145647,138.000000,103.000000,34.302315,0.800000,28.000000,11.000000,13.000000,108.000000,172.000000
4,4,56.000000,4.057940,137.310730,102.666656,32.152634,0.912006,24.481102,13.179167,17.653704,125.283066,211.429657
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,50329.000000,4.300000,139.000000,97.000000,40.600000,1.600000,30.000000,16.000000,32.000000,248.000000,236.222687
9996,9996,50361.000000,3.800000,140.000000,109.000000,34.200000,0.900000,22.000000,13.000000,15.000000,143.000000,174.000000
9997,9997,50362.000000,4.600000,142.000000,107.000000,38.300000,0.900000,26.000000,14.000000,11.000000,138.000000,165.000000
9998,9998,6573.949219,4.800000,139.000000,104.000000,35.400000,0.800000,28.000000,12.000000,7.000000,125.000000,104.000000


In [10]:
y_pred.head()
y_pred = y_pred.rename(columns={
    '1': 'Crt', 
    '2': 'Plt',
    '3':'Hgb',
    '4':'Wbc',
    '5':'Ure',
    '6':'50983',
    '7':'Bic',
    '8':'50971',
    '9':'50902',
    '10':'51221',
    })

y_test = y_test.rename(columns={
    '0': 'Crt', 
    '1': 'Plt',
    '2':'Hgb',
    '3':'Wbc',
    '4':'Ure',
    '5':'50983',
    '6':'Bic',
    '7':'50971',
    '8':'50902',
    '9':'51221',
    })
valid_pos = valid_pos.rename(columns={
    '0': 'Crt', 
    '1': 'Plt',
    '2':'Hgb',
    '3':'Wbc',
    '4':'Ure',
    '5':'50983',
    '6':'Bic',
    '7':'50971',
    '8':'50902',
    '9':'51221',
    })

In [17]:
currTest=y_test['50971']
currImpute=y_pred['50971']
valid_pos = currTest.notna()

In [18]:
mse_pred = mean_squared_error(currTest[valid_pos], currImpute[valid_pos])
r2_pred = r2_score(currTest[valid_pos], currImpute[valid_pos])
itemid="50971"
normalranges = {
  "Crt": (0.7, 1.3),
  "Plt": (150,450),
  "Hgb": (12, 18),
  "Wbc": (4, 11),
  "Ure": (8, 20),
  "50983": (136, 145),
  "Bic": (23, 28),
  "50971": (3.5, 5),
  "50902": (98, 106),
  "51221": (37,50),
}

bins = [-1,normalranges[f"{itemid}"][0],normalranges[f"{itemid}"][1],10000]
# Define labels for the categories
labels = ['Low', 'Medium', 'High']
y_test_cat = pd.cut(currTest[valid_pos],bins=bins, labels=labels).astype(str)
y_pred_cat = pd.cut(currImpute[valid_pos],bins=bins, labels=labels).astype(str)
print(itemid)
print("normal range", bins)
print("xgboost imputation mse ",round(mse_pred,3),"and r2: ", round(r2_pred,3))
cm_pred = confusion_matrix(y_test_cat,y_pred_cat,labels=labels)
print("confusion matrix for xgboost")
print(cm_pred)

50971
normal range [-1, 3.5, 5, 10000]
xgboost imputation mse  19.067 and r2:  0.95
confusion matrix for xgboost
[[  61    0   11]
 [   0  212   15]
 [   0    0 8374]]


In [19]:
currImpute[valid_pos]

0       20.567667
1       19.296064
2       18.000000
3       13.000000
5       12.000000
          ...    
9995    32.000000
9996    15.000000
9997    11.000000
9998     7.000000
9999     6.000000
Name: 50971, Length: 8673, dtype: float64

In [20]:
currTest[valid_pos]

0       19.0
1       22.0
2       18.0
3       13.0
5       12.0
        ... 
9995    32.0
9996    15.0
9997    11.0
9998     7.0
9999     6.0
Name: 50971, Length: 8673, dtype: float64